In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd

In [2]:
def click(x):
    button = driver.find_element(By.XPATH, x)
    button.click()
    time.sleep(2)

def get_bs4(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = bs4(response.text, "html.parser")
        return soup
    else:
        return "problem with the request."

In [7]:
def scraping_with_bs4(product_url_list):
    list_brand,list_product,list_vol,list_price,list_description,list_ingredients = [],[],[],[],[],[]
    n = 0
    for i in tqdm(product_url_list):
        soup = get_bs4(i)
        try:
            brand = soup.find("a",{"class":"rm-product__brand"}).get_text().strip()
        except:
            brand = "" 
        try:
            product = soup.find("div",{"class":"rm-product__title"}).get_text().strip()
        except:
            product = ""
        try:
            vol = soup.find("div",{"class":"rm-product__units"}).find("strong").get_text()
        except:
            vol = ""
        try:
            price = soup.find("div",{"itemprop":"price"})["content"]
        except:
            price = ""
        try:
            description = soup.find("div",{"class":"rm-cms"}).find("p").get_text()
        except:
            description = ""
        try:
            ingredients = soup.find("div",{"id":"GRP_INHALTSSTOFFE"}).find("div",{"class":"rm-accordion__detail-info"}).find("p").get_text()
        except:
            ingredients = ""
        list_brand.append(brand)
        list_product.append(product)
        list_vol.append(vol)
        list_price.append(price)
        list_description.append(description)
        list_ingredients.append(ingredients)
        n+=1
        if n%100 == 0:
            time.sleep(2)

    df = pd.DataFrame()
    df["brand"] = list_brand
    df["product"] = list_product
    df["vol"] = list_vol
    df["price"] = list_price
    df["description"] = list_description
    df["ingredients"] = list_ingredients
    return df

In [8]:
url = "https://www.rossmann.de/de/"
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--headless')
options.add_argument('user-agent=fake-useragent')
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install(),options=options))
driver.maximize_window()
driver.get(url)
time.sleep(3)
accept = ["/html/body/div[4]/div[2]/div/div/div[2]/div/div/button[2]","/html/body/div[5]/div[2]/div/div/div[2]/div/div/button[1]"]
try:
    click(accept[0])
except:
    try:
        click(accept[1])
    except:
        pass
    pass
# time.sleep(2)
makeup = "/html/body/div[2]/header/div[1]/nav/div/div[6]/ul/li[3]/div/span/a"
click(makeup)
# time.sleep(1)
alles = "/html/body/div[2]/header/div[1]/nav/div/div[6]/ul/li[3]/div/div/div[2]/ul/div/span/a"
click(alles)
product_url_list = []
while True:
    try:
        html = driver.page_source
        soup = bs4(html, 'html.parser')
        grid = soup.find("div",{"class":"rm-category__products"})
        content = grid.find_all("div",{"class":"rm-grid__content"})
        for i in content:
            link = i.find("a").get("href")
            link = "https://www.rossmann.de"+link
            product_url_list.append(link)
        next = "/html/body/div[2]/div[1]/div/main/div[2]/div/div/div[2]/div/div[2]/div/div[2]/div[3]/div[3]/ul/li[7]"
        click(next)
        time.sleep(2)
    except:
        break

df = scraping_with_bs4(product_url_list)
df

100%|██████████| 408/408 [12:30<00:00,  1.84s/it]


,brand,product,vol,price,description,ingredients
0,Maybelline New York,Instant Perfector Glow 4-in-1 Make-Up 01 Light,20 ml,12.00,Der Instant Perfector Glow – das 4-in-1 Make-U...,AQUA / WATER • ISOHEXADECANE • DIMETHICONE • G...
1,Maybelline New York,Instant Perfector Glow 4-in-1 Make-Up 1.5 Ligh...,20 ml,12.00,Der Instant Perfector Glow – das 4-in-1 Make-U...,AQUA / WATER • ISOHEXADECANE • DIMETHICONE • G...
2,Maybelline New York,Instant Anti-Age Effekt Concealer Nr. 01 Light,"6,8 ml",8.00,Maybelline New York Instant Anti Age Effekt De...,"Aqua/Water, Cyclopentasiloxane, Dimethicone, I..."
3,Maybelline New York,Fit Me! Concealer Nr. 10 Light,"6,8 ml",5.60,Der Maybelline New York Fit Me! Concealer deck...,"Aqua/Water, Cyclopentasiloxane, Hydrogenated P..."
4,Maybelline New York,Fit Me! Matte + Poreless Puder Nr. 120 Classic...,9 g,5.60,*Verbrauchertest 61 Personen,"Talc, Perlite Triisostearin, Phenyl Trimethico..."
...,...,...,...,...,...,...
403,Catrice,Shine Bomb Lip Lacquer 030 Sweet Talker,3 ml,4.49,Flüssiger Lippenstift mit langanhaltendem Farb...,"ISODODECANE, OCTYLDODECANOL, ALCOHOL, POLYGLYC..."
404,Catrice,Glam & Doll Boost Lash Growth Volume Mascara 010,8 ml,4.99,Voluminöse Wimpern und gleichzeitiges Aktivier...,"AQUA (WATER), SYNTHETIC BEESWAX, GLYCERIN, STE..."
405,FOR YOUR Beauty,Standspiegel 3-fache Vergrößerung,1 Stück,3.49,Der for your Beauty Standspiegel überzeugt dur...,
406,RIVAL loves me,Eyeshadow Palette 01 latest nudes,14 g,4.99,Eyeshadow Palette mit 18 Puderlidschatten für ...,"#1 Ingredients: Mica, Talc, Ethylhexyl Stearat..."
